# Une droite de régression avec la méthode de descente de gradient

Dernièrement, vous avez calculé une droite de régression avec deux méthodes :
- les moindres carrés ;
- l’équation normale.

Une autre méthode, adaptable à des problèmes plus complexes que celui qui nous concerne, consiste à effectuer la recherche des meilleurs paramètres par tâtonnement en commençant par des valeurs aléatoires. Les paramètres s’ajustent au fur et à mesure que la fonction de coût, généralement l’erreur quadratique moyenne (MSE), diminue. On appelle cette méthode la descente de gradient.

## Pré-requis

Sans rentrer dans les détails, la descente de gradient nécessite trois paramètres :
- des valeurs aléatoires pour initialiser $\theta$ ($m$ et $b$) ;
- un nombre d’itérations suffisant pour converger ;
- un taux d’apprentissage, noté $\eta$, qui ne soit ni trop faible, afin d’éviter de ralentir l’entraînement, ni trop élevé, afin d’éviter le sur-entraînement.

En plus de ces paramètres, la descente de gradient ne peut se calculer que sur des **données standardisées**. Effectuons tout d’abord une copie des données sur les manchots :

In [ ]:
import pandas as pd
import numpy as np


# csv file
df = pd.read_csv("../data/penguin-census.csv")

# data
coords = df.loc[:,["body_mass_g","flipper_length_mm"]]
coords = coords.dropna()
coords = coords.to_numpy()

# matrices with coords
X = np.c_[coords[:, 0]]
Y = np.c_[coords[:, 1]]

## Standardisation des données

Avant tout, utilisons la classe `StandardScaler` pour centrer-réduire nos données :

In [ ]:
from sklearn.preprocessing import StandardScaler

# scaler: Z score normalization
scaler = StandardScaler()

# scaling
X_scaled = scaler.fit_transform(X)
Y_scaled = scaler.fit_transform(Y)

## Insertion d’une dimension neutre

Comme lors du calcul de la droite de régression avec l’équation normale, et parce que la descente de gradient implique au final un produit matriciel avec une matrice de dimensions $(2, 1)$ il est nécessaire de rajouter une dimension aux coordonnées sur l’axe des abscisses ($X$) :

In [ ]:
# x0 = 1
X_scaled = np.c_[
    np.ones(X.shape),
    X_scaled
]

## Paramètres obligatoires

Fournissons à présent des valeurs aux paramètres obligatoires de la descente de gradient :

In [ ]:
# learning rate
eta = 0.1

# number of iterations
n_steps = 1000

# rows in X
m = len(X_scaled)

# random values to initialize theta
theta = np.random.randn(2, 1)

## Résolution de la formule

Appliquons maintenant la formule :

$$\theta = \theta - \eta \nabla_\theta \text{MSE}(\theta)$$

In [ ]:
# for each step
for step in range(n_steps):
    # the gradients
    gradients = 2/m * X_scaled.T.dot(X_scaled.dot(theta) - Y_scaled)
    # theta
    theta = theta - eta * gradients

Afin de s’assurer que les valeurs de $\theta$ sont correctes, comparons-les avec la solution obtenue grâce à l’équation normale :

In [ ]:
theta_norm = np.linalg.inv(X_scaled.T.dot(X_scaled)).dot(X_scaled.T).dot(Y_scaled)

print(
    theta[0].round(10) == theta_norm[0].round(10),
    theta[1].round(10) == theta_norm[1].round(10),
    sep="\n"
)

Et pour une vérification graphique :

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# slope and intercept
m = theta[1]
b = theta[0]

# points
X = list(map(float, X_scaled[:, 1]))
Y = list(map(float, Y_scaled))

# predictions
Y_pred = [ float(m * x + b) for x in X ]

# vizualisation
ax = plt.subplots()

ax = sns.lineplot(x=X, y=Y_pred, color="fuchsia")
ax = sns.scatterplot(x=X, y=Y, color="seagreen")

ax.set(xlabel="Body mass (g)", ylabel="Flipper length (mm)")

sns.despine()

plt.show()